In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor()
])

In [4]:
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False, num_workers=2)

In [6]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False, num_workers=2)

In [7]:
model = torchvision.models.resnet18(weights=None, num_classes=10)
criterion = nn.CrossEntropyLoss()

In [17]:
from torch.cuda.amp import GradScaler, autocast

model_c = model.to(device)
optimizer = optim.Adam(model_c.parameters(), lr=0.001)
scaler = GradScaler()

C:\Users\user\AppData\Local\Temp\ipykernel_17428\3335665096.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [18]:
start_time = time.time()

for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels =  data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        
        with autocast():
            outputs = model_c(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
    print(f"AMP Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

end_time = time.time()
print(f"Training time AMP: {end_time - start_time:.2f} seconds\n")

C:\Users\user\AppData\Local\Temp\ipykernel_17428\1053339448.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


AMP Epoch 1, Loss: 0.3934
AMP Epoch 2, Loss: 0.2764
Training time AMP: 31.82 seconds

